In [ ]:
from imutils.perspective import four_point_transform
from matplotlib import pyplot as plt
from imutils import contours
import pytesseract
from skimage.measure import find_contours
import numpy as np
import argparse
import os
import imutils
import cv2
from skimage.morphology import skeletonize, thin
import skimage.io as io
from scipy import stats
import arabic_reshaper
import math


def show_images(images,titles=None):
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [ ]:
#save the image to cells Directory
def Save_img(imageo,Name):
    cv2.imwrite(os.path.join("./Cells" , Name), imageo)

In [ ]:
# resize the image and return it ans 1,0
def ResizeData(Fulldata):
    dim = (100, 100)
    resized = cv2.resize(Fulldata,dim,interpolation = cv2.INTER_CUBIC)
    graycell = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    threadhold = cv2.threshold(graycell,0,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return threadhold


# save all id numbers to numbers folder
def SaveAllNumbers(folder_dir):
    i = 0
    for image_name in os.listdir(folder_dir):
        print("Start : "+folder_dir+image_name)
        image = cv2.imread(folder_dir+image_name,1)
        cells,_ = getCellsofImg(image)
        for row in (cells[1:]):
            cell = removeCellBlackBorder(row[0])
            cutten = getIdonly(cell)
            # show_images([cell, cutten],["cell","cutten"])
            chars = getchars(cutten)
            for char in chars:
                char= ResizeData(char)
                # show_images([char],["char"])
                cv2.imwrite(os.path.join("./Numbers" , str(i)+".jpg" ), char)
                i+=1
        print("Done : "+folder_dir+image_name)

# save all id WrittenNumbers to numbers folder
def SaveAllWrittenNumbers(folder_dir):
    i = 0
    for image_name in os.listdir(folder_dir):
        print("Start : "+folder_dir+image_name)
        image = cv2.imread(folder_dir+image_name,1)
        cells,_ = getCellsofImg(image)
        for row in (cells[1:]):
            cell = removeCellBlackBorder(row[3])
            cutten = getIdonly(cell)
            cutten= ResizeData(cutten)
            cv2.imwrite(os.path.join("./HandWrittenNumber" , str(i)+".jpg" ), cutten)
            i+=1
        print("Done : "+folder_dir+image_name)

# save all id Written Symbols to numbers folder
def SaveAllWrittenSymbols(folder_dir):
    i = 0
    for image_name in os.listdir(folder_dir):
        print("Start : "+folder_dir+image_name)
        image = cv2.imread(folder_dir+image_name,1)
        cells,NumberofColums = getCellsofImg(image)
        for row in (cells[1:]):
            for j in range(4,NumberofColums):
                cell = removeCellBlackBorder(row[j])
                cutten = getSymbolOnly(cell)
                cutten= ResizeData(cutten)
                cv2.imwrite(os.path.join("./HandWrittenSymbols" , str(i)+".jpg" ), cutten)
                i+=1
        print("Done : "+folder_dir+image_name)


In [ ]:
def highlight_cells(val):
    color = 'red' if val == '   ' else 'white'
    return 'background-color: {}'.format(color)

def PrintSheetOfImg(image,image_name,automated_ocr_id,handwritted_ocr_id):
    
    df = pd.DataFrame({})
    cells,NumberofColums = getCellsofImg(image)
    ids=[]
    for id in cells[0][1:]:
        idonly = removeCellBlackBorder(id)
        if(not automated_ocr_id):
            cutten = getIdonly(idonly)
            chars = getchars(cutten)
            id=""
            ResizedChars=[]
            for char in chars:
                char = ResizeData(char)
                ResizedChars.append(char)
            labels=classify_unlabelled_directory(ResizedChars)
            strId = "".join(labels)
            ids.append(strId)
        else:
            idAutomatedValue = getId(idonly)
            ids.append(idAutomatedValue)

    df["Code"]=ids

    ArabicNames=[]

    for arabicName in cells[1][1:]:
        english_name = getId(arabicName,"Arabic")
        Reversing = arabic_reshaper.reshape(english_name)
        Reversing = Reversing.rstrip()
        ArabicNames.append(Reversing)

    df["Arabic Name"]=ArabicNames


    EnglishNames=[]
    for englishName in cells[2][1:]:
        english_name = getId(englishName)
        EnglishNames.append(english_name)

    df["English NAme"]=EnglishNames


    CloumData=[]
    symbolIndex=3
    for symbol in cells[symbolIndex][1:]:
        SymbolOnly = removeCellBlackBorder(symbol)
        if(not handwritted_ocr_id):
            cuttenSymbol = getSymbolOnly(SymbolOnly)
            resizeCuttenSymbol = ResizeData(cuttenSymbol)
            labels=classify_unlabelled_directory([resizeCuttenSymbol])  
            predictedLabels=predictChars(labels)
            CloumData.append(predictedLabels[0])
        else:
            handWrittenAutomatedValue = getId(SymbolOnly)
            CloumData.append(handWrittenAutomatedValue)


    df["1"]=CloumData
    
    for symbolIndex in range(4,len(cells)):
        CloumData=[]
        for symbol in cells[symbolIndex][1:]:
            SymbolOnly = removeCellBlackBorder(symbol)
            cuttenSymbol = getSymbolOnly(SymbolOnly)
            resizeCuttenSymbol = ResizeData(cuttenSymbol)
            labels=classify_unlabelled_directory([resizeCuttenSymbol])  
            predictedLabels=predictChars(labels)
            CloumData.append(predictedLabels[0])
        df[symbolIndex-3]=CloumData

    df = df.replace(['r'], '   ')
    df = df.style.applymap(highlight_cells)

    writer = pd.ExcelWriter("sheet"+image_name+".xlsx", engine='xlsxwriter')
    df.to_excel(writer,sheet_name="sheet"+image_name+".xlsx")
    writer.close()

def PrintSheetOfDir(folder_dir,automated_ocr_id,handwritted_ocr_id ):
    for image_name in os.listdir(folder_dir):
        image = cv2.imread(folder_dir+image_name,1)
        PrintSheetOfImg(image,image_name,automated_ocr_id,handwritted_ocr_id)

folder_dir = "./grade sheet/gradesheet/"
automated_ocr_id=False
automated_ocr_handWritted=False
PrintSheetOfDir(folder_dir,automated_ocr_id,automated_ocr_handWritted )